In [34]:
# !pip uninstall --user openai
# !pip install --user openai 
# !pip install rich
from rich import print

In [1]:
import pandas as pd
import os 

df = pd.read_excel(f'proposals_top20.xlsx', engine='openpyxl')
print(df.shape, df.columns)
df.iloc[0]

(17006, 33) Index(['id', 'end', 'app', 'ipfs', 'type', 'body', 'link', 'title', 'start',
       'state', 'votes', 'author', 'symbol', 'quorum', 'scores', 'created',
       'updated', 'network', 'plugins', 'choices', 'privacy', 'flagged',
       'snapshot', 'strategies', 'validation', 'discussion', 'quorumType',
       '__typename', 'scores_state', 'scores_total', 'scores_updated',
       'scores_by_strategy', 'space_id'],
      dtype='object')


id                    0x910b732e87993804b27bb93f9635d424f36fe760d043...
end                                                          1717156800
app                                                            snapshot
ipfs                  bafkreicdrg6eo5onxxsmjsd35ie2n4rjgjpj63jsolr2o...
type                                                      single-choice
body                  \n\n# tl;dr\n- Lista DAO is a liquid staking a...
link                  https://snapshot.org/#/cakevote.eth/proposal/0...
title                 (UPDATED) Proposal for Lista DAO ($LISTA) IFO ...
start                                                        1716897600
state                                                            closed
votes                                                               299
author                       0xa7551aBe0A066555cb5d859849426fB55543Ca25
symbol                                                         CAKEVOTE
quorum                                                          

In [ ]:
intro = """
1. Governance
- Inclusive: encourage participation from stakeholders
- Restrictive: limit decision-making to select group
- Security: enhance security of governance
- ex) Delegation
2. Business
- Collaboration, Scalability, Token Interoperability
    - ex) Cross chain
- Tokenomics
    - ex: LST, Airdrop, User Pool Expansion 
- Oracle implementation
- Propose new business initiative
- Business implementation
3. Operations
- Strategy for increase fee & revenue
- Compensation management
- Ensure transparency in business operation
4. Finance
- Manage treasury & reinvest fund
- Establish safety or insurance fund
- Ensure transparency in financial operation
5. Other
- Donation
- request money
- off topic
- Education fund
"""

In [64]:
from collections import OrderedDict
from dataclasses import dataclass

intro = """
1. Governance
- Inclusive: encourage participation from stakeholders
- Restrictive: limit decision-making to select group
- Security: enhance security of governance
- ex) Delegation
2. Business
- 
- Collaboration, Scalability, Token Interoperability
    - ex) Cross chain
- Tokenomics
    - ex) LST, Airdrop, User Pool Expansion 
- Oracle implementation
- Propose new business initiative
3. Operations
- Strategy for increase fee & revenue
- Compensation management
- Ensure transparency in business operation
4. Finance
- Manage treasury & reinvest fund
- Establish safety or insurance fund
- Ensure transparency in financial operation
5. Other
- Donation
- request money
- off topic
- Education fund
"""

@dataclass
class Prompt:
    system: tuple[tuple] = (
        ("command",      "Classify proposal type & reason: [Governance, Business, Operation, Finance, Other]"),
        ("template",     "Type:[]\nReason:[ option ]")
    )
    user: tuple[list] = (
        ["space",       ""],
        ["title",       ""],
        ["body",     ""]
    )

    def get_system_prompt(self):
        return "\n".join([f"<{v[0]}>: {v[1]}" for v in self.system])
    
    def get_user_prompt(self):
        return "\n".join([f"<{v[0]}>: {v[1]}" for v in self.user if v[1] != ""])

prompt = Prompt()
print(prompt.get_system_prompt())
print(prompt.get_user_prompt())

<command>: Classify proposal type & reason: [Governance, Business, Operation, Finance, Other]
<template>: Type:[]
Reason:[ option ]

In [65]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

message = [
    {
      "role": "system",
      "content": prompt.get_system_prompt()
    },
    {
      "role": "assistant",
      "content": intro
    },
    {
      "role": "user",
      "content": ""
    }
  ]

def get_response(user_input:str):
    message[1]['content'] = user_input

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=message,
        temperature=1,
        max_tokens=100,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response #.choices[0].message['content'][0]['text']


In [71]:
proposal = df.iloc[0]

prompt.user[0][1] = proposal['space_id']
prompt.user[1][1] = proposal['title']
prompt.user[2][1] = "" #proposal['body']

response = get_response(prompt.get_user_prompt())

In [72]:
print(prompt.get_user_prompt())
print(response)

<space>: cakevote.eth
<title>: (UPDATED) Proposal for Lista DAO ($LISTA) IFO to be Hosted on PancakeSwap

ChatCompletion(
    id='chatcmpl-9qvlPq35Ipb2TDaeK2U8hv4bKi5gQ',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Type: [Governance]  \nReason: [This proposal relates to decision-making within the Lista 
DAO concerning the hosting of an Initial Farm Offering (IFO) on a decentralized platform, which aligns with 
governance activities.]',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1722402127,
    model='gpt-4o-mini-2024-07-18',
    object='chat.completion',
    service_tier=None,
    system_fingerprint='fp_0f03d4f0ee',
    usage=CompletionUsage(completion_tokens=42, prompt_tokens=76, total_tokens=118)
)

In [67]:
print(response)

ChatCompletion(
    id='chatcmpl-9qvjCdHB7r45Ru4hYdp6lPvlF3iyu',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Type: [Governance]\nReason: [The proposal relates to a governance decision regarding the 
hosting of an Initial Farm Offering (IFO) for the LISTA DAO project on PancakeSwap, which requires community voting
and input.]',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1722401990,
    model='gpt-4o-mini-2024-07-18',
    object='chat.completion',
    service_tier=None,
    system_fingerprint='fp_0f03d4f0ee',
    usage=CompletionUsage(completion_tokens=45, prompt_tokens=1836, total_tokens=1881)
)

In [63]:
print(response)

ChatCompletion(
    id='chatcmpl-9qvi7pSAqSQNdqJmRfXA5M15vZ2le',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Type:[Governance]\nReason:[This proposal involves the voting process for the IFO (Initial 
Farm Offering) of the LISTA token, which requires community governance participation to make decisions regarding 
the project.]',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1722401923,
    model='gpt-4o-mini-2024-07-18',
    object='chat.completion',
    service_tier=None,
    system_fingerprint='fp_0f03d4f0ee',
    usage=CompletionUsage(completion_tokens=40, prompt_tokens=1836, total_tokens=1876)
)

In [43]:
print(response)

ChatCompletion(
    id='chatcmpl-9quMEtPzc3U8UKcHSEObSrW9UvKRx',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Type: Business  \nReason: This proposal outlines a plan for an Initial Farm Offering (IFO)
to raise funds for the Lista DAO project. It details the mechanisms for participation, token economics, and the 
overarching business strategy for liquidity and governance within the decentralized finance space.',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1722396722,
    model='gpt-4o-mini-2024-07-18',
    object='chat.completion',
    service_tier=None,
    system_fingerprint='fp_0f03d4f0ee',
    usage=CompletionUsage(completion_tokens=53, prompt_tokens=1846, total_tokens=1899)
)

In [39]:
print(response)

ChatCompletion(
    id='chatcmpl-9qu21aJaKbwvBnZDgoKgWIjRphbnn',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Type: Business  \nReason: This proposal describes the hosting of an Initial Farm Offering 
(IFO) for LISTA, detailing the structure of the sale, token distribution, and participation methods, which are 
elements focused on business collaboration, scalability, and tokenomics within the DeFi space.',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1722395469,
    model='gpt-4o-mini-2024-07-18',
    object='chat.completion',
    service_tier=None,
    system_fingerprint='fp_0f03d4f0ee',
    usage=CompletionUsage(completion_tokens=56, prompt_tokens=2012, total_tokens=2068)
)

In [35]:
print(response)

ChatCompletion(
    id='chatcmpl-9qtuhWAKybJWeh5bCLXBw8hPQAPu9',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Type: [Business]  \nReason: The proposal outlines the hosting of an Initial Farm Offering 
(IFO) for Lista DAO, detailing tokenomics, governance features, and the method of raising funds through public and 
private sales, which falls under business initiatives and token interoperability in the DeFi space.',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1722395015,
    model='gpt-4o-mini-2024-07-18',
    object='chat.completion',
    service_tier=None,
    system_fingerprint='fp_611b667b19',
    usage=CompletionUsage(completion_tokens=58, prompt_tokens=2065, total_tokens=2123)
)